In [32]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time, re, pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin

BASE_URL = "https://www.yodobashi.com"
START_URL = "https://www.yodobashi.com/category/19531/"
SLEEP_SEC = 3.0
TARGET_COUNT = 5  # ← 取得件数
EXCEL_NAME = "yodobashi_all_products.xlsx"

# ✅ Chrome設定
options = Options()
# options.add_argument("--headless")  # 実ブラウザで確認する場合はコメントアウト
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

service = Service()
driver = webdriver.Chrome(service=service, options=options)

def extract_product_links(page_html):
    """商品リンクを抽出"""
    soup = BeautifulSoup(page_html, "html.parser")
    links = [urljoin(BASE_URL, a["href"]) for a in soup.select("a[href^='/product/']")]
    return list(dict.fromkeys(links))  # 重複削除

def get_product_info(url):
    """個別ページから商品情報を取得（在庫関係なし）"""
    driver.get(url)
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "h1"))
    )
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 商品タイトル
    title_tag = soup.select_one("h1")
    title = title_tag.get_text(strip=True) if title_tag else "不明"

    # 型番（タイトル中の半角英数字を抽出）
    model = None
    match = re.search(r"[A-Z0-9\-]{3,}", title)
    if match:
        model = match.group()

    # 価格取得
    price_tag = soup.select_one(".productPrice__main") or soup.select_one(".productPrice")
    price = None
    if price_tag:
        price_text = re.sub(r"[^\d]", "", price_tag.get_text())
        price = int(price_text) if price_text.isdigit() else None

    # JANコード
    jan = None
    for dl in soup.select("div.specTbl dl"):
        dt = dl.find("dt")
        dd = dl.find("dd")
        if dt and dd and "JAN" in dt.get_text():
            jan = dd.get_text(strip=True)
            break

    # 在庫テキストも一応取得（参考用）
    stock_tag = soup.select_one(".salesInfoTxt")
    stock = stock_tag.get_text(strip=True) if stock_tag else ""

    return {
        "title": title,
        "model": model,
        "price": price,
        "jan": jan,
        "stock": stock,
        "url": url,
    }

# ---- メイン処理 ----
results = []
page_num = 1
stop_flag = False

print(f"🌸 ヨドバシカテゴリ巡回開始: {START_URL}\n")

try:
    while not stop_flag:
        list_url = START_URL + f"?page={page_num}"
        print(f"\n📄 カテゴリページ {page_num} を取得中...")
        driver.get(list_url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        links = extract_product_links(str(soup))
        print(f"🔗 商品リンク {len(links)}件")

        if not links:
            print("⚠️ 商品リンクが見つかりません。終了。")
            break

        for link in links:
            if len(results) >= TARGET_COUNT:
                stop_flag = True
                break

            try:
                data = get_product_info(link)
                if data:
                    results.append(data)
                    # 🌟リアルタイム表示
                    print("────────────────────────────")
                    print(f"✅ {len(results):03d}件目")
                    print(f"📦 商品名: {data['title']}")
                    print(f"🔢 型番: {data['model']}")
                    print(f"💴 価格: {data['price']}円")
                    print(f"📇 JAN: {data['jan']}")
                    print(f"📦 在庫欄の文字: {data['stock']}")
                    print(f"🌐 URL: {data['url']}")
                    print("────────────────────────────\n")

            except Exception as e:
                print("⚠️ 商品取得エラー:", e)

            time.sleep(SLEEP_SEC)

        page_num += 1

finally:
    driver.quit()

# ---- Excel保存 ----
if results:
    df = pd.DataFrame(results)
    df.to_excel(EXCEL_NAME, index=False)
    print(f"\n✅ 商品 {len(results)} 件を取得完了！")
    print(f"📊 Excel出力完了 → {EXCEL_NAME}")
else:
    print("⚠️ 1件も取得できませんでした。")


🌸 ヨドバシカテゴリ巡回開始: https://www.yodobashi.com/category/19531/


📄 カテゴリページ 1 を取得中...
🔗 商品リンク 167件
────────────────────────────
✅ 001件目
📦 商品名: 明治 meiji ほほえみ明治 ほほえみ らくらくキューブ 1620g [赤ちゃん用 0ヶ月～1歳頃]
🔢 型番: 1620
💴 価格: 6200円
📇 JAN: None
📦 在庫欄の文字: 
🌐 URL: https://www.yodobashi.com/product/100000001008037083/
────────────────────────────

────────────────────────────
✅ 002件目
📦 商品名: ユニ･チャーム ムーニームーニーおしりふき やわらか素材 詰替 76枚×8個
🔢 型番: None
💴 価格: 1010円
📇 JAN: None
📦 在庫欄の文字: 
🌐 URL: https://www.yodobashi.com/product/100000001007482885/
────────────────────────────

────────────────────────────
✅ 003件目
📦 商品名: セガフェイブDREAM SWITCH（ドリームスイッチ）ベーシックセット 対象年齢：3歳～
🔢 型番: DREAM
💴 価格: 9350円
📇 JAN: None
📦 在庫欄の文字: 
🌐 URL: https://www.yodobashi.com/product/100000001009314695/
────────────────────────────

────────────────────────────
✅ 004件目
📦 商品名: タカラトミー TAKARATOMYプラレール キミのまちをうごかそう！プラレールベストセレクションセット [対象年齢：3歳～]
🔢 型番: TAKARATOMY
💴 価格: 4590円
📇 JAN: None
📦 在庫欄の文字: 
🌐 URL: https://www.yodobashi.com/product/100000001008968679/
─────

In [40]:
import os
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

BASE_URL = "https://www.yodobashi.com"
START_URL = "https://www.yodobashi.com/category/19531/"
SLEEP_SEC = 3.0
TARGET_COUNT = 5  # ← 取得件数

# ✅ 保存先をデスクトップに変更
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
EXCEL_NAME = os.path.join(desktop_path, "yodobashi_model_price.xlsx")

# ✅ Chrome設定
options = Options()
# options.add_argument("--headless")  # 実ブラウザ確認時はコメントアウト
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

service = Service()
driver = webdriver.Chrome(service=service, options=options)

# ✅ 除外したいメーカー名一覧
MAKER_KEYWORDS = {
    "SONY", "SHARP", "TOSHIBA", "HITACHI", "PANASONIC", "NEC", "FUJITSU", "CANON", "EPSON",
    "NIKON", "OLYMPUS", "PENTAX", "CASIO", "ASUS", "ACER", "HP", "DELL", "LENOVO", "MSI",
    "LOGICOOL", "BUFFALO", "ELECOM", "IODATA", "SANDISK", "KINGSTON", "ADATA",
    "YAMAHA", "KORG", "ROLAND", "BOSE", "DENON", "PIONEER", "KENWOOD", "ONKYO",
    "DAIKIN", "CORONA", "MITSUBISHI", "IRIS", "OHYAMA", "Dyson",
    "MEIJI", "ユニチャーム", "タカラトミー", "SEGA", "BANDAI", "NINTENDO",
    "シャープ", "パナソニック", "ソニー", "ニコン", "キヤノン", "エプソン",
    "サンディスク", "アイリス", "富士通", "東芝", "日立", "三菱", "象印", "タイガー"
}

# ✅ 除外したい単位（数字の後ろについていたら除外）
UNIT_SUFFIXES = [
    "G", "KG", "ML", "L", "枚", "個", "本", "箱", "mL", "cc", "cm", "mm", "GB", "TB"
]

def extract_product_links(page_html):
    """商品リンクを抽出"""
    soup = BeautifulSoup(page_html, "html.parser")
    links = [urljoin(BASE_URL, a["href"]) for a in soup.select("a[href^='/product/']")]
    return list(dict.fromkeys(links))  # 重複削除

def extract_model_from_title(title):
    """タイトルからメーカー名と単位を除外して型番候補を抽出"""
    # スペース・全角スペース・記号で分割
    words = re.split(r"[\s　,，、・\[\]\(\)【】／/＋＋]", title)
    candidates = []

    for w in words:
        w = w.strip()
        if not w:
            continue
        # アルファベット・数字・ハイフン・アンダーバーのみ
        if re.match(r"^[A-Za-z0-9\-\_]+$", w):
            upper_w = w.upper()

            # メーカー名リストに含まれず
            if upper_w in MAKER_KEYWORDS:
                continue

            # 英字＋数字を両方含み
            if not (re.search(r"[A-Za-z]", w) and re.search(r"\d", w)):
                continue

            # 数字＋単位（例：1620g, 500ml）を除外
            if re.match(r"^\d+(?:\.\d+)?(" + "|".join(UNIT_SUFFIXES) + r")$", upper_w):
                continue

            # 数字で始まって単位が続くものも除外（例: 10L, 50ML）
            if any(upper_w.endswith(u) for u in UNIT_SUFFIXES):
                continue

            candidates.append(w)

    # 最も長いものを型番とみなす（SR-C01M-W など）
    if candidates:
        return max(candidates, key=len)
    return None

def get_product_info(url):
    """個別ページから商品情報を取得"""
    driver.get(url)
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1")))
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # --- 商品タイトル ---
    title_tag = soup.select_one("h1")
    title = title_tag.get_text(strip=True) if title_tag else "不明"

    # --- 型番抽出 ---
    model = extract_model_from_title(title)

    # --- 価格 ---
    price_tag = soup.select_one(".productPrice__main") or soup.select_one(".productPrice")
    price = None
    if price_tag:
        price_text = re.sub(r"[^\d]", "", price_tag.get_text())
        price = int(price_text) if price_text.isdigit() else None

    return {
        "title": title,
        "model": model,
        "price": price,
        "url": url,
    }

# ---- メイン処理 ----
results = []
page_num = 1
stop_flag = False

print(f"🌸 ヨドバシカテゴリ巡回開始: {START_URL}\n")

try:
    while not stop_flag:
        list_url = START_URL + f"?page={page_num}"
        print(f"\n📄 カテゴリページ {page_num} を取得中...")
        driver.get(list_url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        links = extract_product_links(str(soup))
        print(f"🔗 商品リンク {len(links)}件")

        if not links:
            print("⚠️ 商品リンクが見つかりません。終了。")
            break

        for link in links:
            if len(results) >= TARGET_COUNT:
                stop_flag = True
                break

            try:
                data = get_product_info(link)
                if data:
                    results.append(data)
                    # 🌟リアルタイム表示
                    print("────────────────────────────")
                    print(f"✅ {len(results):03d}件目")
                    print(f"📦 商品名: {data['title']}")
                    print(f"🔢 型番: {data['model']}")
                    print(f"💴 価格: {data['price']}円")
                    print(f"🌐 URL: {data['url']}")
                    print("────────────────────────────\n")

            except Exception as e:
                print("⚠️ 商品取得エラー:", e)

            time.sleep(SLEEP_SEC)

        page_num += 1

finally:
    driver.quit()

# ---- Excel保存 ----
if results:
    df = pd.DataFrame(results)
    df.to_excel(EXCEL_NAME, index=False)
    print(f"\n✅ 商品 {len(results)} 件を取得完了！")
    print(f"📊 Excel出力完了 → {EXCEL_NAME}")
else:
    print("⚠️ 1件も取得できませんでした。")


🌸 ヨドバシカテゴリ巡回開始: https://www.yodobashi.com/category/19531/


📄 カテゴリページ 1 を取得中...
🔗 商品リンク 167件
────────────────────────────
✅ 001件目
📦 商品名: 明治 meiji ほほえみ明治 ほほえみ らくらくキューブ 1620g [赤ちゃん用 0ヶ月～1歳頃]
🔢 型番: None
💴 価格: 6200円
🌐 URL: https://www.yodobashi.com/product/100000001008037083/
────────────────────────────

────────────────────────────
✅ 002件目
📦 商品名: ユニ･チャーム ムーニームーニーおしりふき やわらか素材 詰替 76枚×8個
🔢 型番: None
💴 価格: 1010円
🌐 URL: https://www.yodobashi.com/product/100000001007482885/
────────────────────────────

────────────────────────────
✅ 003件目
📦 商品名: セガフェイブDREAM SWITCH（ドリームスイッチ）ベーシックセット 対象年齢：3歳～
🔢 型番: None
💴 価格: 9350円
🌐 URL: https://www.yodobashi.com/product/100000001009314695/
────────────────────────────

────────────────────────────
✅ 004件目
📦 商品名: タカラトミー TAKARATOMYプラレール キミのまちをうごかそう！プラレールベストセレクションセット [対象年齢：3歳～]
🔢 型番: None
💴 価格: 4590円
🌐 URL: https://www.yodobashi.com/product/100000001008968679/
────────────────────────────

────────────────────────────
✅ 005件目
📦 商品名: シャープ SHARPSR-C01M-W [対話AIキャラクタ

In [41]:
import os
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

BASE_URL = "https://www.yodobashi.com"
START_URL = "https://www.yodobashi.com/category/19531/19532/"  # このカテゴリ
SLEEP_SEC = 2.5
TARGET_COUNT = 10  # テスト取得件数

# 保存先（デスクトップ）
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
EXCEL_NAME = os.path.join(desktop_path, "yodobashi_pc_peripherals.xlsx")

# Chrome設定
options = Options()
# options.add_argument("--headless")  # 必要に応じて有効化
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

service = Service()
driver = webdriver.Chrome(service=service, options=options)

# メーカー除外リスト（例）
MAKER_KEYWORDS = {
    "SONY", "SHARP", "PANASONIC", "APPLE", "DELL", "HP", "ASUS", "ACER",
    "BUFFALO", "ELECOM", "LOGICOOL", "IODATA", "ADATA", "SAN DISK", "SANDISK",
    "日本語メーカー名", "富士通", "東芝", "ソニー", "シャープ", "パナソニック"
}

UNIT_SUFFIXES = ["G", "KG", "ML", "L", "枚", "個", "本", "箱", "mL", "cc", "cm", "mm", "GB", "TB"]

def extract_product_links(page_html):
    """カテゴリページから商品リンクを取得"""
    soup = BeautifulSoup(page_html, "html.parser")
    # js_productListPostTag を含む a タグ、または href に /product/ を含む a タグ
    anchors = soup.select("a.js_productListPostTag[href^='/product/'], a[href^='/product/']")
    links = [urljoin(BASE_URL, a["href"]) for a in anchors if a.get("href")]
    return list(dict.fromkeys(links))

def extract_model_from_title(title):
    """タイトルから型番候補を抽出（メーカー名除外・単位除外含む）"""
    words = re.split(r"[\s　,，、・\[\]\(\)【】／/＋\+]+", title)
    candidates = []
    for w in words:
        w = w.strip()
        if not w:
            continue
        # 英数字とハイフン・アンダースコアのみ
        if re.match(r"^[A-Za-z0-9\-\_]+$", w):
            upper_w = w.upper()
            # メーカー名リストに含まれるものは除外
            if upper_w in MAKER_KEYWORDS:
                continue
            # 英字と数字の両方含むもの
            if not (re.search(r"[A-Za-z]", w) and re.search(r"\d", w)):
                continue
            # 単位付き数字 (例: 1620G, 500ML) を除外
            if any(upper_w.endswith(u) for u in UNIT_SUFFIXES):
                continue
            # 型番らしいパターン（例: ABC-123, XYZ1234, A1B2C3 など）
            if re.match(r"^[A-Z0-9]{2,5}([-_][A-Z0-9]{1,5})?$", upper_w):
                candidates.append(w)
    if candidates:
        return max(candidates, key=len)
    return None

def get_product_info(url):
    """個別商品のタイトル・型番・価格を取得"""
    driver.get(url)
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1")))
    soup = BeautifulSoup(driver.page_source, "html.parser")

    title_tag = soup.select_one("h1")
    title = title_tag.get_text(strip=True) if title_tag else "不明"

    model = extract_model_from_title(title)

    price_tag = soup.select_one(".productPrice__main") or soup.select_one(".productPrice")
    price = None
    if price_tag:
        price_text = re.sub(r"[^\d]", "", price_tag.get_text())
        if price_text.isdigit():
            price = int(price_text)

    return {"title": title, "model": model, "price": price, "url": url}

# メイン処理
results = []
page_num = 1
stop_flag = False

print(f"カテゴリ巡回開始: {START_URL}")

try:
    while not stop_flag:
        list_url = START_URL + f"?page={page_num}"
        print(f"\n📄 カテゴリページ {page_num} を取得中...")
        driver.get(list_url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        links = extract_product_links(str(soup))
        print(f"🔗 商品リンク {len(links)} 件")

        if not links:
            print("⚠️ 商品リンクが見つかりません。終了。")
            break

        for link in links:
            if len(results) >= TARGET_COUNT:
                stop_flag = True
                break
            try:
                data = get_product_info(link)
                results.append(data)
                print("────────────────────────────")
                print(f"✅ {len(results):03d}件目")
                print(f"📦 商品名: {data['title']}")
                print(f"🔢 型番: {data['model']}")
                print(f"💴 価格: {data['price']} 円")
                print(f"🌐 URL: {data['url']}")
                print("────────────────────────────\n")
            except Exception as e:
                print("⚠️ 取得エラー:", e)
            time.sleep(SLEEP_SEC)

        page_num += 1

finally:
    driver.quit()

# Excel出力
if results:
    df = pd.DataFrame(results)
    df.to_excel(EXCEL_NAME, index=False)
    print(f"\n✅ 完了：{len(results)} 件取得 → {EXCEL_NAME}")
else:
    print("⚠️ 出力できる商品はありませんでした。")


カテゴリ巡回開始: https://www.yodobashi.com/category/19531/19532/

📄 カテゴリページ 1 を取得中...
🔗 商品リンク 179 件
────────────────────────────
✅ 001件目
📦 商品名: 明治 meiji ほほえみ明治 ほほえみ らくらくキューブ 1620g [赤ちゃん用 0ヶ月～1歳頃]
🔢 型番: None
💴 価格: 6200 円
🌐 URL: https://www.yodobashi.com/product/100000001008037083/
────────────────────────────

────────────────────────────
✅ 002件目
📦 商品名: ユニ･チャーム ムーニームーニーおしりふき やわらか素材 詰替 76枚×8個
🔢 型番: None
💴 価格: 1010 円
🌐 URL: https://www.yodobashi.com/product/100000001007482885/
────────────────────────────

────────────────────────────
✅ 003件目
📦 商品名: セガフェイブDREAM SWITCH（ドリームスイッチ）ベーシックセット 対象年齢：3歳～
🔢 型番: None
💴 価格: 9350 円
🌐 URL: https://www.yodobashi.com/product/100000001009314695/
────────────────────────────

────────────────────────────
✅ 004件目
📦 商品名: タカラトミー TAKARATOMYプラレール キミのまちをうごかそう！プラレールベストセレクションセット [対象年齢：3歳～]
🔢 型番: None
💴 価格: 4590 円
🌐 URL: https://www.yodobashi.com/product/100000001008968679/
────────────────────────────

────────────────────────────
✅ 005件目
📦 商品名: シャープ SHARPSR-C01M-W [対話AIキ

In [47]:
import os
import re
import time
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 🔍 「家電」検索結果URL（ヨドバシ）
SEARCH_URL = "https://www.yodobashi.com/?word=家電"
BASE_URL = "https://www.yodobashi.com"
TARGET_COUNT = 100  # ← 取得件数
SLEEP_SEC = 0.8     # ← ページ間の待機（0.8秒で十分）

# ✅ 出力先（デスクトップ）
desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
EXCEL_NAME = os.path.join(desktop_path, "yodobashi_kaden.xlsx")

# ✅ Chrome設定（headless OFFで安定）
options = Options()
# options.add_argument("--headless")  # ← 初回はOFFで安定動作確認
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--blink-settings=imagesEnabled=false")  # 画像非読み込み
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.set_page_load_timeout(45)  # ← タイムアウトを45秒に延長

MAKER_KEYWORDS = {
    "SONY", "SHARP", "PANASONIC", "APPLE", "ANKER", "JBL", "BOSE",
    "ELECOM", "BUFFALO", "LOGICOOL", "HITACHI", "TOSHIBA"
}
UNIT_SUFFIXES = ["G", "KG", "ML", "L", "枚", "個", "本", "箱", "mL", "GB", "TB"]

def extract_model_from_title(title):
    """タイトルから型番候補を抽出"""
    words = re.split(r"[\s　,，、・\[\]\(\)【】／/＋\+]", title)
    candidates = []
    for w in words:
        w = w.strip()
        if not w:
            continue
        if re.match(r"^[A-Za-z0-9\-\_]+$", w):
            upper_w = w.upper()
            if upper_w in MAKER_KEYWORDS:
                continue
            if not (re.search(r"[A-Za-z]", w) and re.search(r"\d", w)):
                continue
            if any(upper_w.endswith(u) for u in UNIT_SUFFIXES):
                continue
            if re.match(r"^[A-Z0-9]{2,5}[-_]?[A-Z0-9]{2,6}([-_][A-Z0-9]{1,3})?$", upper_w):
                candidates.append(w)
    return max(candidates, key=len) if candidates else None

def collect_links():
    """検索結果ページから商品リンクを全件収集（複数ページ対応）"""
    driver.get(SEARCH_URL)
    time.sleep(3)
    all_links = []
    page = 1

    while len(all_links) < TARGET_COUNT:
        soup = BeautifulSoup(driver.page_source, "html.parser")
        anchors = soup.select("a.js_productListPostTag[href^='/product/']")
        links = [urljoin(BASE_URL, a["href"]) for a in anchors]
        for l in links:
            if l not in all_links:
                all_links.append(l)
        print(f"📄 ページ{page}: {len(links)}件 (合計 {len(all_links)}件)")
        # 次のページを探す
        next_btn = soup.select_one("a.next")
        if not next_btn or "disabled" in next_btn.get("class", []):
            break
        next_url = urljoin(BASE_URL, next_btn["href"])
        driver.get(next_url)
        page += 1
        time.sleep(2)

    return all_links[:TARGET_COUNT]

def get_product_info(url):
    """個別商品ページから情報取得"""
    try:
        driver.get(url)
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "h1")))
        soup = BeautifulSoup(driver.page_source, "html.parser")
        title_tag = soup.select_one("h1")
        title = title_tag.get_text(strip=True) if title_tag else "不明"
        model = extract_model_from_title(title)

        price_tag = soup.select_one(".productPrice__main") or soup.select_one(".productPrice")
        price = None
        if price_tag:
            price_text = re.sub(r"[^\d]", "", price_tag.get_text())
            price = int(price_text) if price_text.isdigit() else None

        return {"title": title, "model": model, "price": price, "url": url}
    except Exception:
        return None

# ---- メイン処理 ----
start = time.time()
print("🌸 ヨドバシ『家電』検索結果から商品を取得中...\n")

results = []
links = collect_links()

for i, link in enumerate(links, 1):
    data = get_product_info(link)
    if data:
        results.append(data)
        print(f"✅ {i:03d}/{len(links)}: {data['title'][:30]} | {data['model']} | ￥{data['price']}")
    time.sleep(SLEEP_SEC)

driver.quit()

if results:
    df = pd.DataFrame(results)
    df.to_excel(EXCEL_NAME, index=False)
    elapsed = time.time() - start
    print(f"\n✅ {len(results)}件取得完了！（{elapsed:.1f}秒）")
    print(f"📊 出力 → {EXCEL_NAME}")
else:
    print("⚠️ データが取得できませんでした。")


🌸 ヨドバシ『家電』検索結果から商品を取得中...

📄 ページ1: 136件 (合計 92件)
📄 ページ2: 138件 (合計 185件)
✅ 001/100: パナソニック Panasonic洗濯槽クリーナー N-W1A | None | ￥1740
✅ 002/100: パナソニック Panasonic洗濯槽クリーナー N-W1A | None | ￥1740
✅ 003/100: パナソニック Panasonicドラム式洗濯機用洗濯槽クリー | None | ￥1320
✅ 004/100: パナソニック Panasonicドラム式洗濯機用洗濯槽クリー | None | ￥1320
✅ 005/100: 日立 HITACHI洗濯槽クリーナー SK-1500 | SK-1500 | ￥2140
✅ 006/100: 日立 HITACHI洗濯槽クリーナー SK-1500 | SK-1500 | ￥2140
✅ 007/100: パナソニック Panasonicボディトリマー 充電式 黒  | ER-GK83-S | ￥9250
✅ 008/100: パナソニック Panasonicボディトリマー 充電式 黒  | ER-GK83-S | ￥9250
✅ 009/100: ブラウン BRAUN専用洗浄液 クリーン&リニューシステム専 | CCR5CR | ￥4980
✅ 010/100: ブラウン BRAUN専用洗浄液 クリーン&リニューシステム専 | CCR5CR | ￥4980
✅ 011/100: 東レ TORAYPTC.SV2J [トレビーノ ポット型浄水 | None | ￥3040
✅ 012/100: 東レ TORAYPTC.SV2J [トレビーノ ポット型浄水 | None | ￥3040
✅ 013/100: パナソニック Panasonic単機能レンジ フラットタイプ | NE-FL1C-W | ￥16470
✅ 014/100: パナソニック Panasonic単機能レンジ フラットタイプ | NE-FL1C-W | ￥16470
✅ 015/100: パナソニック Panasonicシェーバー洗浄充電器 専用洗 | ES-4L03 | ￥886
✅ 016/100: パナソニック Panasonicシェーバー洗浄充電器 専用洗 | ES-

In [6]:
import requests
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote

# ---------- 設定 ----------
INPUT_FILE = r"C:\Users\taku5\Desktop\yodobashi_kaden.xlsx"       # 読み込むExcel（1列目にJAN）
OUTPUT_FILE = "toysrus_prices.xlsx"  # 出力ファイル名
BASE_URL = "https://www.toysrus.co.jp/search/?q="

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/118.0 Safari/537.36"
}

# ---------- 価格取得関数 ----------
def get_toysrus_price(jan: str):
    try:
        url = BASE_URL + quote(jan)
        resp = requests.get(url, headers=HEADERS, timeout=15)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, "html.parser")

        # 商品名（検索結果 or 詳細ページ）
        title_tag = soup.select_one(".product-title a, h1.product-title")
        title = title_tag.get_text(strip=True) if title_tag else "商品名不明"

        # 価格を抽出
        price_tag = soup.select_one(".price, .sales-price, .product-sales-price")
        price = None
        if price_tag:
            text = price_tag.get_text()
            price = "".join([c for c in text if c.isdigit()])
            if price == "":
                price = None

        if price:
            return title, int(price)
        else:
            return title, None

    except Exception as e:
        return f"取得失敗: {e}", None


# ---------- メイン処理 ----------
def main():
    try:
        df = pd.read_excel(INPUT_FILE, header=None)
    except Exception as e:
        print(f"❌ Excel読み込みエラー: {e}")
        return

    results = []
    total = len(df)

    print(f"📘 {total}件のJANコードをトイザらスで処理します...\n")

    for i, row in df.iterrows():
        jan = str(row[0]).strip()
        if not jan or jan.lower() == "nan":
            continue

        print(f"🔍 {i+1}/{total} 検索中: {jan} ...")
        title, price = get_toysrus_price(jan)

        if price:
            print(f"✅ {title} | ￥{price}\n")
        else:
            print(f"⚠️ 価格取得失敗: {title}\n")

        results.append({"JAN": jan, "商品名": title, "価格": price or "取得失敗"})

        # ランダムウェイトで安全アクセス
        time.sleep(random.uniform(4, 8))

    # 保存
    output_df = pd.DataFrame(results)
    output_df.to_excel(OUTPUT_FILE, index=False)
    print(f"\n🎉 完了！結果を {OUTPUT_FILE} に保存しました。")


if __name__ == "__main__":
    main()
"""
janからトイザらス比較
"""


📘 2件のJANコードをトイザらスで処理します...

🔍 1/2 検索中: 4904810000099 ...
✅ 商品名不明 | ￥6929

🔍 2/2 検索中: 4904810990154 ...
✅ 商品名不明 | ￥1399


🎉 完了！結果を toysrus_prices.xlsx に保存しました。


In [21]:
import requests
import random

API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
DOMAIN_JP = 5
ELECTRONICS_CATEGORY_ID = 637394  # 家電カテゴリ

def fetch_profitable_prime_electronics():
    payload = {
        "selection": {
            "domain": DOMAIN_JP,
            "rootCategory": ELECTRONICS_CATEGORY_ID,
            "perPage": 3,               # 複数候補を確認したい場合
            "page": 0,
            "isPrimeAvailable": True,   # ✅ プライム販売あり
            "hasAmazonOffer": False,    # ✅ Amazon本体なし
            "sellerCount_NEW_lte": 5,   # ✅ 新品出品者5人以下
            "offerCountFBA_NEW_gte": 1, # ✅ FBA在庫あり（現在も販売中）
            "salesRankDrops90_gte": 20, # ✅ 90日で20回以上売れた（売れ筋）
            "salesRankL90DaysAverage_lte": 10000,  # ✅ 上位カテゴリ
            "current_NEW_lte": 30000,   # ✅ 新品価格3万円以下
        }
    }

    finder_url = f"https://api.keepa.com/query?key={API_KEY}&domain={DOMAIN_JP}"
    resp = requests.post(finder_url, json=payload)
    data = resp.json()

    tokens_used = data.get("tokensConsumed")
    tokens_left = data.get("tokensLeft")
    print(f"🔹 Finder消費: {tokens_used} / 残り: {tokens_left}")

    asin_list = data.get("asinList", [])
    if not asin_list:
        print("❌ 該当商品が見つかりませんでした。")
        return

    # ランダムに1件抽出
    asin = random.choice(asin_list)
    print(f"✅ 抽出ASIN: {asin}")

    # Product APIで詳細を取得
    prod_url = f"https://api.keepa.com/product?key={API_KEY}&domain={DOMAIN_JP}&asin={asin}&history=0"
    prod_resp = requests.get(prod_url)
    prod_data = prod_resp.json()

    p = prod_data["products"][0]
    title = p.get("title", "タイトル不明")
    jan = p.get("ean") or (p.get("eanList")[0] if p.get("eanList") else "JAN不明")
    price = (p.get("buyBoxPrice") or p.get("currentOfferPrice")) or "不明"
    rank = p.get("salesRankDrops90", "不明")

    print("📦 商品タイトル:", title)
    print("🔸 JANコード:", jan)
    print("💰 価格:", price)
    print("📊 売れ筋指標（90日内変動）:", rank)
    print("✅ 完了")

if __name__ == "__main__":
    fetch_profitable_prime_electronics()


🔹 Finder消費: 11 / 残り: 289
✅ 抽出ASIN: B0CDPGCLBR
📦 商品タイトル: EYESRAIL Boxx Guardian ポケモンカード用 BOX ローダー UVカット 国内製造 コレクション 透明 保管ケース (デッキシールド, 【10個】64枚用)
🔸 JANコード: 4595640126686
💰 価格: 不明
📊 売れ筋指標（90日内変動）: 不明
✅ 完了


In [23]:
import requests, random

API_KEY = "5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j"
DOMAIN_JP = 5
CATEGORY_ID = 637394   # ✅ イヤホン・ヘッドホン

def fetch_japanese_audio_items():
    payload = {
        "selection": {
            "domain": DOMAIN_JP,
            "rootCategory": CATEGORY_ID,
            "perPage": 3,
            "page": random.randint(0, 5),  # 🔄 ページをランダム化
            "isPrimeAvailable": True,
            "hasAmazonOffer": False,
            "sellerCount_NEW_lte": 5,
            "offerCountFBA_NEW_gte": 1,
            "salesRankDrops90_gte": 20,
            "salesRankL90DaysAverage_lte": 10000,
            "current_NEW_lte": 20000,  # 2万円以下
            # 日本メーカー名で絞り込み（AND条件）
            "title": {
                "include": ["ソニー", "パナソニック", "オーディオテクニカ", "ビクター", "JVC", "KENWOOD"]
            }
        }
    }

    url = f"https://api.keepa.com/query?key={API_KEY}&domain={DOMAIN_JP}"
    resp = requests.post(url, json=payload)
    data = resp.json()

    print(f"🔹 Finder消費: {data.get('tokensConsumed')} / 残り: {data.get('tokensLeft')}")

    asin_list = data.get("asinList", [])
    if not asin_list:
        print("❌ 該当商品が見つかりません。")
        return

    asin = random.choice(asin_list)
    print(f"✅ 抽出ASIN: {asin}")

    prod_url = f"https://api.keepa.com/product?key={API_KEY}&domain={DOMAIN_JP}&asin={asin}&history=0"
    prod_resp = requests.get(prod_url)
    prod_data = prod_resp.json()

    p = prod_data["products"][0]
    title = p.get("title", "タイトル不明")
    jan = p.get("ean") or (p.get("eanList")[0] if p.get("eanList") else "JAN不明")
    price = (p.get("buyBoxPrice") or p.get("currentOfferPrice")) or "不明"

    print("📦 商品タイトル:", title)
    print("🔸 JANコード:", jan)
    print("💰 価格:", price)
    print(f"🔗 https://www.amazon.co.jp/dp/{asin}")
    print("✅ 完了！")

if __name__ == "__main__":
    fetch_japanese_audio_items()


🔹 Finder消費: 11 / 残り: 282
✅ 抽出ASIN: B0F9KKRYBG
📦 商品タイトル: VTCOSMETICS(ブイティコスメテックス) フェイスマスク スキンケア (9.クライオ ひんやりマスク)
🔸 JANコード: 8803463012700
💰 価格: 不明
🔗 https://www.amazon.co.jp/dp/B0F9KKRYBG
✅ 完了！


In [6]:
import pyautogui
import time
import os

# -------------------------
# 設定
# -------------------------
import os


# 🎨 画像フォルダ
IMG_DIR = r"C:\Users\taku5\Desktop\toy"

# 🔍 各画像ファイル
SEARCH_ICON = os.path.join(IMG_DIR, "kensaku.png")  # 虫眼鏡ボタンなど
SEARCH_ALL = os.path.join(IMG_DIR, "all.png")        # 商品一覧の部分
SEARCH_BAR = os.path.join(IMG_DIR, "mado.png")       # 検索窓

for path in [SEARCH_ICON, SEARCH_BAR, PRODUCT_THUMB]:
    print(os.path.exists(path), path)


SEARCH_WORD = "セール"
MAX_PRODUCTS = 5
CONFIDENCE = 0.8

# -------------------------
# 検索実行
# -------------------------
def search_products():
    # 虫眼鏡クリック
    icon = pyautogui.locateCenterOnScreen(SEARCH_ICON, confidence=CONFIDENCE)
    if not icon:
        print("⚠️ 虫眼鏡アイコンが見つかりません。画面を最前面にしてください。")
        return False
    pyautogui.click(icon)
    time.sleep(1)

    # 入力欄検出
    search_box = pyautogui.locateCenterOnScreen(SEARCH_BAR, confidence=CONFIDENCE)
    if not search_box:
        print("⚠️ 検索バーが見つかりません。")
        return False

    pyautogui.click(search_box)
    pyautogui.write(SEARCH_WORD, interval=0.05)
    pyautogui.press("enter")
    print(f"🔍 『{SEARCH_WORD}』で検索しました。")
    time.sleep(5)
    return True

# -------------------------
# 商品巡回
# -------------------------
def browse_products():
    thumbs = list(pyautogui.locateAllOnScreen(PRODUCT_THUMB, confidence=CONFIDENCE))
    if not thumbs:
        print("⚠️ 商品画像が見つかりません。")
        return
    print(f"🔗 {len(thumbs)}件の商品を検出。上位{MAX_PRODUCTS}件を開きます。")

    for i, pos in enumerate(thumbs[:MAX_PRODUCTS], 1):
        pyautogui.click(pos)
        print(f"✅ {i}件目を開きました。")
        time.sleep(3)
        pyautogui.hotkey("alt", "left")  # ← 戻る
        time.sleep(2)

# -------------------------
# 実行
# -------------------------
print("🚀 トイザらス 自動巡回テスト開始\n")
time.sleep(3)  # 実行前にブラウザを最前面へ

if search_products():
    browse_products()

print("\n🏁 テスト完了！")


True C:\Users\taku5\Desktop\toy\kensaku.png
True C:\Users\taku5\Desktop\toy\mado.png
False C:\Users\taku5\Desktop\toy\タイトルなし.png
🚀 トイザらス 自動巡回テスト開始



ImageNotFoundException: 

In [14]:
import pyautogui, pyperclip, os, time

IMG_DIR = r"C:\Users\taku5\Desktop\toy"
SEARCH_ICON = os.path.join(IMG_DIR, "kensaku.png")
SEARCH_BAR = os.path.join(IMG_DIR, "mado.png")

# 虫眼鏡クリック
pos = pyautogui.locateOnScreen(SEARCH_ICON, confidence=0.8)
if pos:
    pyautogui.click(pyautogui.center(pos))
    print("✅ 虫眼鏡クリック完了")
else:
    print("⚠️ 虫眼鏡が見つかりません。")

time.sleep(2)

# 検索窓にフォーカスして「セール」を貼り付け
bar = pyautogui.locateOnScreen(SEARCH_BAR, confidence=0.7)
if bar:
    pyautogui.click(pyautogui.center(bar))
    time.sleep(0.5)

    pyperclip.copy("セール")          # クリップボードにコピー
    pyautogui.hotkey("ctrl", "v")     # 貼り付け
    pyautogui.press("enter")          # 検索実行

    print("✅ 『セール』を貼り付けて検索しました！")
else:
    print("⚠️ 検索窓が見つかりません。")


✅ 虫眼鏡クリック完了
✅ 『セール』を貼り付けて検索しました！


In [17]:
import pyautogui, pyperclip, os, time, sys

# ==============================
# 設定
# ==============================
IMG_DIR = r"C:\Users\taku5\Desktop\toy"
SEARCH_ICON = os.path.join(IMG_DIR, "kensaku.png")   # 虫眼鏡アイコン
SEARCH_BAR = os.path.join(IMG_DIR, "mado.png")       # 検索窓
PRICE_IMG  = os.path.join(IMG_DIR, "math.png")       # 赤い価格（例：¥1,997）

# ==============================
# 関数定義
# ==============================

def click_search_icon():
    """虫眼鏡クリック"""
    pos = pyautogui.locateOnScreen(SEARCH_ICON, confidence=0.8)
    if pos:
        pyautogui.click(pyautogui.center(pos))
        print("✅ 虫眼鏡クリック完了")
    else:
        print("⚠️ 虫眼鏡が見つかりません。")

def input_search_keyword():
    """検索窓に『セール』を入力"""
    bar = pyautogui.locateOnScreen(SEARCH_BAR, confidence=0.7)
    if bar:
        pyautogui.click(pyautogui.center(bar))
        time.sleep(0.5)
        pyperclip.copy("セール")
        pyautogui.hotkey("ctrl", "v")
        pyautogui.press("enter")
        print("✅ 『セール』を貼り付けて検索しました！")
    else:
        print("⚠️ 検索窓が見つかりません。")

def find_and_click_prices():
    """ページ内の赤い価格を順にクリックして戻る"""
    found_any = False
    for pos in pyautogui.locateAllOnScreen(PRICE_IMG, confidence=0.8):
        found_any = True
        x, y = pyautogui.center(pos)
        click_y = y - 80   # 👈 価格より上（約80px上）をクリック
        pyautogui.moveTo(x, click_y, duration=0.2)
        pyautogui.click()
        print(f"🛒 商品クリック → {x}, {click_y}（価格より上）")
        time.sleep(3)  # 商品ページ読み込み待機
        pyautogui.hotkey("alt", "left")  # 戻る
        time.sleep(2)
    return found_any

def scroll_down():
    """ページを下にスクロール"""
    pyautogui.scroll(-800)
    time.sleep(2)

# ==============================
# 実行
# ==============================
print("🔍 セール商品検索から開始します...")
click_search_icon()
time.sleep(2)
input_search_keyword()
time.sleep(5)

print("🚀 セール商品の自動巡回を開始します...")

while True:
    found = find_and_click_prices()
    if not found:
        scroll_down()
        again = pyautogui.locateOnScreen(PRICE_IMG, confidence=0.8)
        if not again:
            print("🎉 最後まで到達しました。終了します。")
            sys.exit()


🔍 セール商品検索から開始します...
✅ 虫眼鏡クリック完了
✅ 『セール』を貼り付けて検索しました！
🚀 セール商品の自動巡回を開始します...
🛒 商品クリック → 1980, 942（価格より上）
🛒 商品クリック → 2451, 942（価格より上）
🛒 商品クリック → 2921, 942（価格より上）
🛒 商品クリック → 3392, 942（価格より上）
🛒 商品クリック → 1980, 1584（価格より上）
🛒 商品クリック → 2451, 1584（価格より上）
🛒 商品クリック → 2921, 1584（価格より上）
🛒 商品クリック → 3392, 1584（価格より上）
🛒 商品クリック → 1980, 942（価格より上）
🛒 商品クリック → 2451, 942（価格より上）



KeyboardInterrupt



In [59]:
import pyautogui, pyperclip, os, time, sys, random

# ==============================
# 設定（ここを変えればすぐ調整可能）
# ==============================
IMG_DIR = r"C:\Users\taku5\Desktop\toy"
SEARCH_ICON = os.path.join(IMG_DIR, "kensaku.png")
SEARCH_BAR = os.path.join(IMG_DIR, "mado.png")
PRICE_IMG  = os.path.join(IMG_DIR, "math.png")

SCROLL_MIN = 900   # 👈 スクロールの最小値（大きくすると速く進む）
SCROLL_MAX = 1400  # 👈 スクロールの最大値

# ==============================
# 関数定義
# ==============================
def human_sleep(base: float, var: float = 0.6):
    """人間っぽいランダムな待機時間"""
    time.sleep(base + random.uniform(-var, var))

def human_move(x, y, base_duration=0.5):
    """マウスを少しブレながら動かす"""
    x += random.randint(-5, 5)
    y += random.randint(-5, 5)
    duration = base_duration + random.uniform(0, 0.4)
    pyautogui.moveTo(x, y, duration=duration)

def click_search_icon():
    pos = pyautogui.locateOnScreen(SEARCH_ICON, confidence=0.8)
    if pos:
        x, y = pyautogui.center(pos)
        human_move(x, y)
        pyautogui.click()
        print("✅ 虫眼鏡クリック完了")
        human_sleep(1)
    else:
        print("⚠️ 虫眼鏡が見つかりません。")

def input_search_keyword():
    bar = pyautogui.locateOnScreen(SEARCH_BAR, confidence=0.7)
    if bar:
        x, y = pyautogui.center(bar)
        human_move(x, y)
        pyautogui.click()
        human_sleep(0.8)
        pyperclip.copy("セール")
        pyautogui.hotkey("ctrl", "v")
        human_sleep(0.5)
        pyautogui.press("enter")
        print("✅ 『セール』検索完了")
    else:
        print("⚠️ 検索窓が見つかりません。")

def find_and_click_prices(clicked_positions):
    found_any = False
    positions = list(pyautogui.locateAllOnScreen(PRICE_IMG, confidence=0.8))
    random.shuffle(positions)  # 順番をランダムにして自然な動きに

    for pos in positions:
        x, y = pyautogui.center(pos)

        if any(abs(x - px) < 10 and abs(y - py) < 10 for px, py in clicked_positions):
            continue

        found_any = True
        click_y = y - random.randint(60, 100)  # 上の位置も少しバラつかせる
        human_move(x, click_y, base_duration=0.4)
        pyautogui.click()
        print(f"🛒 商品クリック → {x}, {click_y}")
        clicked_positions.append((x, y))

        human_sleep(random.uniform(2.5, 4.5))
        pyautogui.hotkey("alt", "left")
        human_sleep(random.uniform(2, 3.5))

    return found_any

def scroll_down():
    """ページを下にスクロール（ランダム幅で自然に）"""
    step = random.randint(SCROLL_MIN, SCROLL_MAX)
    pyautogui.scroll(-step)
    print(f"⬇️ スクロール：{step}px")
    human_sleep(random.uniform(1.5, 3.0))

# ==============================
# 実行
# ==============================
print("🔍 セール商品検索から開始します...")
click_search_icon()
human_sleep(2)
input_search_keyword()
human_sleep(5)

print("🚀 セール商品の自動巡回を開始します...")

clicked_positions = []
empty_scrolls = 0

while True:
    found = find_and_click_prices(clicked_positions)
    if not found:
        empty_scrolls += 1
        scroll_down()
        if empty_scrolls >= 3:
            print("🎉 ページ末尾まで到達。終了します。")
            sys.exit()
    else:
        empty_scrolls = 0

"""
うまくいった
"""


🔍 セール商品検索から開始します...
✅ 虫眼鏡クリック完了
✅ 『セール』検索完了
🚀 セール商品の自動巡回を開始します...
🛒 商品クリック → 3390, 939
🛒 商品クリック → 2919, 938
🛒 商品クリック → 2919, 1596
🛒 商品クリック → 1978, 1588
🛒 商品クリック → 3390, 1599
🛒 商品クリック → 2449, 920
🛒 商品クリック → 2449, 1588
🛒 商品クリック → 1978, 944
⬇️ スクロール：1043px
🛒 商品クリック → 2449, 935


KeyboardInterrupt: 

In [69]:
import pyautogui
import time
import os
import pyperclip

# ==============================
# 設定
# ==============================
IMG_DIR = r"C:\Users\taku5\Desktop\toy"
YEN_IMG = os.path.join(IMG_DIR, "math2.png")     # 円マーク画像
BARCODE_IMG = r"C:\Users\taku5\Desktop\toy\ba-ko-.png"  # 「バーコード:」画像

SCROLL_STEP = -600   # スクロール量
MAX_SCROLL = 20      # 最大スクロール回数

# ==============================
# 円マーク検出 → コピー
# ==============================
print("🔍 円マークを探しています...")
time.sleep(2)

pos = pyautogui.locateOnScreen(YEN_IMG, confidence=0.7)
if not pos:
    print("⚠️ 円マークが見つかりません。")
    raise SystemExit

x, y = pyautogui.center(pos)
x, y = int(x), int(y)
print(f"✅ 円マーク検出: X={x}, Y={y}")

# 💴 右側をダブルクリックしてコピー
click_x = x + 80
click_y = y
pyautogui.moveTo(click_x, click_y, duration=0.4)
pyautogui.doubleClick()
time.sleep(0.3)
pyautogui.hotkey("ctrl", "c")
time.sleep(0.3)

copied_text = pyperclip.paste().strip()
if copied_text:
    print(f"💰 コピーされた文字列: {copied_text}")
else:
    print("⚠️ コピー内容が空でした。")

# ==============================
# スクロールしながらバーコード検出
# ==============================
print("\n📜 スクロールしながらバーコードを探します...")

found = False
for i in range(MAX_SCROLL):
    try:
        pos = pyautogui.locateOnScreen(BARCODE_IMG, confidence=0.7)
    except pyautogui.ImageNotFoundException:
        pos = None

    if pos:
        x, y = pyautogui.center(pos)
        print(f"✅ バーコード見つかりました: X={x}, Y={y}")
        found = True
        break
    else:
        pyautogui.scroll(SCROLL_STEP)
        print(f"↕️ スクロール {i+1}/{MAX_SCROLL} 回目...（バーコード未検出）")
        time.sleep(1)

if not found:
    print("❌ バーコードが画面内で見つかりませんでした。")
    raise SystemExit

# ==============================
# バーコード右の空白をクリック → 右までドラッグ → コピー
# ==============================
print("\n🖱️ 空白をクリックしたまま右までドラッグしてコピーします...")

# 「バーコード:」の右に少しずらしてクリック開始
start_x = x + 130   # 文字のすぐ右の空白
end_x = start_x + 900  # 右端（やや余裕を持たせる）
drag_y = y           # 同じ高さ

# 🖱️ マウス操作：空白クリック→右までドラッグ
pyautogui.moveTo(start_x, drag_y, duration=0.3)
pyautogui.mouseDown()        # クリックしたまま
pyautogui.moveTo(end_x, drag_y, duration=0.7)  # 右へドラッグ
pyautogui.mouseUp()          # 離す
time.sleep(0.3)

# コピー
pyautogui.hotkey("ctrl", "c")
time.sleep(0.3)
jan_code = pyperclip.paste().strip()

if jan_code:
    print(f"📦 コピーされたバーコード文字: {jan_code}")
else:
    print("⚠️ バーコード文字がコピーできませんでした。")


"""
うまくいった
"""


🔍 円マークを探しています...
✅ 円マーク検出: X=3135, Y=662
💰 コピーされた文字列: 6,997

📜 スクロールしながらバーコードを探します...
↕️ スクロール 1/20 回目...（バーコード未検出）
↕️ スクロール 2/20 回目...（バーコード未検出）
✅ バーコード見つかりました: X=2239, Y=1801

🖱️ 空白をクリックしたまま右までドラッグしてコピーします...
📦 コピーされたバーコード文字: 4904810934851


In [75]:
import pyautogui, pyperclip, os, time, sys, random, csv
from datetime import datetime

# ==============================
# 設定
# ==============================
IMG_DIR = r"C:\Users\taku5\Desktop\toy"
SEARCH_ICON = os.path.join(IMG_DIR, "kensaku.png")
SEARCH_BAR  = os.path.join(IMG_DIR, "mado.png")
PRICE_IMG   = os.path.join(IMG_DIR, "math.png")
YEN_IMG     = os.path.join(IMG_DIR, "math2.png")
BARCODE_IMG = os.path.join(IMG_DIR, "ba-ko-.png")

SCROLL_MIN = 900
SCROLL_MAX = 1400
BARCODE_SCROLL = -600
MAX_BARCODE_SCROLL = 20

OUTPUT_CSV = os.path.join(os.path.expanduser("~/Desktop"), "商品データ.csv")

# ==============================
# 補助関数
# ==============================
def human_sleep(base: float, var: float = 0.6):
    """人間っぽいランダムな待機（0秒未満にならない）"""
    delay = base + random.uniform(-var, var)
    if delay < 0:
        delay = 0.05  # 最低でも0.05秒は待つ
    time.sleep(delay)


def human_move(x, y, base_duration=0.5):
    x += random.randint(-5, 5)
    y += random.randint(-5, 5)
    duration = base_duration + random.uniform(0, 0.4)
    pyautogui.moveTo(x, y, duration=duration)

def scroll_down():
    step = random.randint(SCROLL_MIN, SCROLL_MAX)
    pyautogui.scroll(-step)
    print(f"⬇️ スクロール：{step}px")
    human_sleep(random.uniform(0.8, 1.5))

def save_to_csv(data):
    header = ["取得時刻", "価格", "バーコード"]
    write_header = not os.path.exists(OUTPUT_CSV)
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow(header)
        writer.writerow(data)

# ==============================
# 検索実行
# ==============================
print("🔍 検索開始...")

icon = pyautogui.locateOnScreen(SEARCH_ICON, confidence=0.7)
if icon:
    x, y = pyautogui.center(icon)
    human_move(x, y)
    pyautogui.click()
    print("✅ 虫眼鏡クリック完了")
    human_sleep(1)
else:
    print("⚠️ 虫眼鏡が見つかりません。")

bar = pyautogui.locateOnScreen(SEARCH_BAR, confidence=0.7)
if bar:
    x, y = pyautogui.center(bar)
    human_move(x, y)
    pyautogui.click()
    pyperclip.copy("セール")
    pyautogui.hotkey("ctrl", "v")
    pyautogui.press("enter")
    print("✅ 『セール』検索完了")
else:
    print("⚠️ 検索窓が見つかりません。")

human_sleep(10)
print("🚀 商品ページ巡回開始...")

# ==============================
# 商品ページ巡回
# ==============================
clicked_positions = []
empty_scrolls = 0
product_count = 0

while True:
    positions = list(pyautogui.locateAllOnScreen(PRICE_IMG, confidence=0.7))
    if not positions:
        empty_scrolls += 1
        scroll_down()
        if empty_scrolls >= 3:
            print("🎉 全商品チェック完了！終了します。")
            break
        continue

    empty_scrolls = 0
    random.shuffle(positions)

    for pos in positions:
        x, y = pyautogui.center(pos)
        if any(abs(x - px) < 10 and abs(y - py) < 10 for px, py in clicked_positions):
            continue

        clicked_positions.append((x, y))
        click_y = y - random.randint(60, 100)
        human_move(x, click_y)
        pyautogui.click()
        print(f"🛒 商品ページを開く ({x},{click_y})")
        human_sleep(3)

        # ==============================
        # 円マーク検出 → 価格コピー
        # ==============================
        price_text = ""
        pos2 = pyautogui.locateOnScreen(YEN_IMG, confidence=0.7)
        if pos2:
            x2, y2 = pyautogui.center(pos2)
            pyautogui.moveTo(x2 + 80, y2, duration=0.3)
            pyautogui.doubleClick()
            pyautogui.hotkey("ctrl", "c")
            human_sleep(0.3)
            price_text = pyperclip.paste().strip()
            print(f"💰 価格取得: {price_text}")
        else:
            print("⚠️ 円マークが見つかりません。")

        # ==============================
        # バーコード探索＋コピー
        # ==============================
        jan_code = ""
        for i in range(MAX_BARCODE_SCROLL):
            try:
                bpos = pyautogui.locateOnScreen(BARCODE_IMG, confidence=0.7)
            except pyautogui.ImageNotFoundException:
                bpos = None

            if bpos:
                x3, y3 = pyautogui.center(bpos)
                print(f"✅ バーコード見つかりました: X={x3}, Y={y3}")
                start_x = x3 + 130
                end_x = start_x + 900
                drag_y = y3
                pyautogui.moveTo(start_x, drag_y, duration=0.3)
                pyautogui.mouseDown()
                pyautogui.moveTo(end_x, drag_y, duration=0.7)
                pyautogui.mouseUp()
                pyautogui.hotkey("ctrl", "c")
                human_sleep(0.3)
                jan_code = pyperclip.paste().strip()
                print(f"📦 バーコード取得: {jan_code}")
                break
            else:
                pyautogui.scroll(BARCODE_SCROLL)
                human_sleep(0.8)
        if not jan_code:
            print("⚠️ バーコードが見つかりませんでした。")

        # ==============================
        # CSVに保存
        # ==============================
        if price_text or jan_code:
            product_count += 1
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            save_to_csv([timestamp, price_text, jan_code])
            print(f"🧾 {product_count}件目を保存 ✅")

        # ==============================
        # 戻る
        # ==============================
        pyautogui.hotkey("alt", "left")
        human_sleep(random.uniform(2.5, 4.5))

print(f"\n📁 収集完了！保存先: {OUTPUT_CSV}")


🔍 検索開始...


ImageNotFoundException: 

In [76]:
import pyautogui, pyperclip, os, time, sys, random, csv
from datetime import datetime
import pandas as pd

# ==============================
# 設定
# ==============================
IMG_DIR = r"C:\Users\taku5\Desktop\toy"
SEARCH_ICON = os.path.join(IMG_DIR, "kensaku.png")
SEARCH_BAR  = os.path.join(IMG_DIR, "mado.png")
PRICE_IMG   = os.path.join(IMG_DIR, "math.png")
YEN_IMG     = os.path.join(IMG_DIR, "math2.png")
BARCODE_IMG = os.path.join(IMG_DIR, "ba-ko-.png")

SCROLL_MIN = 900
SCROLL_MAX = 1400
BARCODE_SCROLL = -600
MAX_BARCODE_SCROLL = 20

# 保存先
OUTPUT_DIR = os.path.expanduser("~/Desktop")
CSV_PATH   = os.path.join(OUTPUT_DIR, "商品データ.csv")
XLSX_PATH  = os.path.join(OUTPUT_DIR, "商品データ.xlsx")

# ==============================
# 補助関数
# ==============================
def human_sleep(base: float, var: float = 0.6):
    delay = base + random.uniform(-var, var)
    if delay < 0:
        delay = 0.05
    time.sleep(delay)

def human_move(x, y, base_duration=0.5):
    x += random.randint(-5, 5)
    y += random.randint(-5, 5)
    duration = base_duration + random.uniform(0, 0.4)
    pyautogui.moveTo(x, y, duration=duration)

def scroll_down():
    step = random.randint(SCROLL_MIN, SCROLL_MAX)
    pyautogui.scroll(-step)
    print(f"⬇️ スクロール：{step}px")
    human_sleep(random.uniform(0.8, 1.5))

def save_to_csv(jan_code, price_text):
    """CSVに1行ずつ追記"""
    header = ["商品コード", "価格"]
    write_header = not os.path.exists(CSV_PATH)
    with open(CSV_PATH, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if write_header:
            writer.writerow(header)
        writer.writerow([jan_code, price_text])

# ==============================
# 検索実行
# ==============================
print("🔍 検索開始...")

icon = pyautogui.locateOnScreen(SEARCH_ICON, confidence=0.7)
if icon:
    x, y = pyautogui.center(icon)
    human_move(x, y)
    pyautogui.click()
    print("✅ 虫眼鏡クリック完了")
    human_sleep(1)
else:
    print("⚠️ 虫眼鏡が見つかりません。")

bar = pyautogui.locateOnScreen(SEARCH_BAR, confidence=0.7)
if bar:
    x, y = pyautogui.center(bar)
    human_move(x, y)
    pyautogui.click()
    pyperclip.copy("セール")
    pyautogui.hotkey("ctrl", "v")
    pyautogui.press("enter")
    print("✅ 『セール』検索完了")
else:
    print("⚠️ 検索窓が見つかりません。")

human_sleep(10)
print("🚀 商品ページ巡回開始...")

# ==============================
# 商品ページ巡回
# ==============================
clicked_positions = []
empty_scrolls = 0
product_count = 0

while True:
    try:
        positions = list(pyautogui.locateAllOnScreen(PRICE_IMG, confidence=0.7))
    except pyautogui.ImageNotFoundException:
        positions = []

    if not positions:
        empty_scrolls += 1
        scroll_down()
        if empty_scrolls >= 3:
            print("🎉 全商品チェック完了！Excel出力へ進みます。")
            break
        continue

    empty_scrolls = 0
    random.shuffle(positions)

    for pos in positions:
        x, y = pyautogui.center(pos)
        if any(abs(x - px) < 10 and abs(y - py) < 10 for px, py in clicked_positions):
            continue

        clicked_positions.append((x, y))
        click_y = y - random.randint(60, 100)
        human_move(x, click_y)
        pyautogui.click()
        print(f"🛒 商品ページを開く ({x},{click_y})")
        human_sleep(3)

        # ==============================
        # 円マーク検出 → 価格コピー
        # ==============================
        price_text = ""
        try:
            pos2 = pyautogui.locateOnScreen(YEN_IMG, confidence=0.7)
        except pyautogui.ImageNotFoundException:
            pos2 = None

        if pos2:
            x2, y2 = pyautogui.center(pos2)
            pyautogui.moveTo(x2 + 80, y2, duration=0.3)
            pyautogui.doubleClick()
            pyautogui.hotkey("ctrl", "c")
            human_sleep(0.3)
            price_text = pyperclip.paste().strip()
            print(f"💰 価格取得: {price_text}")
        else:
            print("⚠️ 円マークが見つかりません。")

        # ==============================
        # バーコード探索＋コピー
        # ==============================
        jan_code = ""
        for i in range(MAX_BARCODE_SCROLL):
            try:
                bpos = pyautogui.locateOnScreen(BARCODE_IMG, confidence=0.7)
            except pyautogui.ImageNotFoundException:
                bpos = None

            if bpos:
                x3, y3 = pyautogui.center(bpos)
                print(f"✅ バーコード見つかりました: X={x3}, Y={y3}")
                start_x = x3 + 130
                end_x = start_x + 900
                drag_y = y3
                pyautogui.moveTo(start_x, drag_y, duration=0.3)
                pyautogui.mouseDown()
                pyautogui.moveTo(end_x, drag_y, duration=0.7)
                pyautogui.mouseUp()
                pyautogui.hotkey("ctrl", "c")
                human_sleep(0.3)
                jan_code = pyperclip.paste().strip()
                print(f"📦 バーコード取得: {jan_code}")
                break
            else:
                pyautogui.scroll(BARCODE_SCROLL)
                human_sleep(0.8)
        if not jan_code:
            print("⚠️ バーコードが見つかりませんでした。")

        # ==============================
        # CSVに保存（商品コード → 価格）
        # ==============================
        if price_text or jan_code:
            product_count += 1
            save_to_csv(jan_code, price_text)
            print(f"🧾 {product_count}件目を保存 ✅")

        # ==============================
        # 戻る
        # ==============================
        pyautogui.hotkey("alt", "left")
        human_sleep(random.uniform(2.5, 4.5))

# ==============================
# 最後にExcelファイルに変換
# ==============================
print("📊 Excelファイル（.xlsx）を作成しています...")
df = pd.read_csv(CSV_PATH)
df.to_excel(XLSX_PATH, index=False)

print(f"\n✅ 完了！Excel保存先：{XLSX_PATH}")


🔍 検索開始...
✅ 虫眼鏡クリック完了
✅ 『セール』検索完了
🚀 商品ページ巡回開始...
🛒 商品ページを開く (3390,924)
⚠️ 円マークが見つかりません。
✅ バーコード見つかりました: X=2239, Y=1531
📦 バーコード取得: 4987176206374, 4987176341549
🧾 1件目を保存 ✅
🛒 商品ページを開く (3390,1566)
💰 価格取得: 3,197
✅ バーコード見つかりました: X=2239, Y=1681
📦 バーコード取得: 4894843038407
🧾 2件目を保存 ✅
🛒 商品ページを開く (2449,939)
💰 価格取得: 5,387
✅ バーコード見つかりました: X=2239, Y=1981
📦 バーコード取得: 4987493000358, 4987493000488
🧾 3件目を保存 ✅
🛒 商品ページを開く (2449,1601)
💰 価格取得: 2,997
✅ バーコード見つかりました: X=2239, Y=1531
📦 バーコード取得: 4987176206626
🧾 4件目を保存 ✅
🛒 商品ページを開く (2920,933)
💰 価格取得: 2,997
✅ バーコード見つかりました: X=2239, Y=1411
📦 バーコード取得: 4987176206510, 4987176341327
🧾 5件目を保存 ✅
🛒 商品ページを開く (2920,1573)
💰 価格取得: 2,997
✅ バーコード見つかりました: X=2239, Y=1411
📦 バーコード取得: 4987176206220, 4987176341204
🧾 6件目を保存 ✅
🛒 商品ページを開く (1978,1586)
💰 価格取得: 2,997
✅ バーコード見つかりました: X=2239, Y=1411
📦 バーコード取得: 4987176206329, 4987176341259
🧾 7件目を保存 ✅
🛒 商品ページを開く (1978,938)
💰 価格取得: 2,997
✅ バーコード見つかりました: X=2239, Y=1531
📦 バーコード取得: 4987176206206, 4987176341532
🧾 8件目を保存 ✅


OSError: screen grab failed

In [ ]:
5evt1mqp5d7ju3q7kmlt8s27lp0gf8n51oird2ivf6b8oj1ko5s2ltnd2n9dgo9j